In [2]:
contest_folder = 'practice'   # CHANGE THIS
contest_folder = 'contestData'
contest_folder = 'round1'

In [3]:
from analysis import get_current_status, select_solution

In [4]:
df_grouped, aggregated_df = get_current_status(contest_folder)

In [5]:
aggregated_list = aggregated_df.to_dict(orient='records')

In [6]:
# df_grouped

In [7]:
aggregated_df

,problem_code,problem_name,statement,sample_in,sample_out,full_in,solution_id,execution_response,execution_code,execution_sample_out,execution_full_out,case_string,case_string_found,sample_out_found,hash,status,timestring
0,subsonic_subway,Subsonic Subway,"When he's not busy fighting with Dr. Robotnik,...",4\n3\n0 10\n0 10\n0 10\n4\n0 3\n1 4\n1 2\n0 10...,Case #1: 0.3\nCase #2: 1.5\nCase #3: -1\nCase ...,111\n3\n0 10\n0 10\n0 10\n4\n0 3\n1 4\n1 2\n0 ...,"[000, 001]",[**Key Findings:**\n\n1. **Understanding the P...,[import sys\n\ndef main():\n import sys\n ...,[Case #1: 0.3\nCase #2: 1.5\nCase #3: -1\nCase...,[Case #1: 0.3\nCase #2: 1.5\nCase #3: -1\nCase...,"[Case #111, Case #111]","[True, True]","[True, False]",40ad7bc7,not_analyzed,4643
1,substantial_losses,Substantial Losses,"You currently weigh $W$ units, but you would i...",5\n201 200 1\n185 183 2\n250 180 0\n7766554433...,Case #1: 3\nCase #2: 10\nCase #3: 70\nCase #4:...,114\n201 200 1\n185 183 2\n250 180 0\n77665544...,"[000, 001]",[**Key Findings:**\n\n1. **Understanding the P...,[import sys\n\nMOD = 998244353\n\ndef main():\...,[Case #1: 3\nCase #2: 10\nCase #3: 70\nCase #4...,[Case #1: 3\nCase #2: 10\nCase #3: 70\nCase #4...,"[Case #114, Case #114]","[True, True]","[True, True]",3b8f5638,not_analyzed,4643
2,substitution_cipher,Substitution Cipher,You’ve probably seen the most classic substitu...,6\n??2 3\n135201 1\n?35 2\n1?0 2\n1122 1\n3???...,Case #1: 122 3\nCase #2: 135201 2\nCase #3: 13...,128\n??2 3\n135201 1\n?35 2\n1?0 2\n1122 1\n3?...,"[001, 002]",[To solve this problem efficiently within the ...,[import sys\nimport threading\nMOD = 998244353...,[Case #1: 116\nCase #2: 135201 2\nCase #3: 13...,[Case #1: 116\nCase #2: 135201 2\nCase #3: 13...,"[Case #128, Case #128]","[True, True]","[False, False]",8efece30,not_analyzed,4643


In [8]:
import time
import threading

while True:
    _, aggregated_df = get_current_status(contest_folder)
    if len(aggregated_df) == 0:
        print("Not outputs found yet")
        time.sleep(30)
        continue
    
    aggregated_list = aggregated_df.to_dict(orient='records')
    print(f"Running analysis at {aggregated_df['timestring'][0]}")
    
    for row in aggregated_list:
        if row["status"] != "not_analyzed":
            continue
        analysis_thread = threading.Thread(target=select_solution, args=(row,))
        analysis_thread.start()
        print(f"Running analysis at {row['timestring']} for {row['problem_name']}")

    time.sleep(30)

Running analysis at 4644
Running analysis at 4644 for Subsonic Subway
Running analysis at 4644 for Substantial Losses
Running analysis at 4644 for Substitution Cipher
Did not select solution for substitution_cipher, forcing any solution.
Selected solution 001 for substitution_cipher
Selected solution 001 for substantial_losses
Selected solution 000 for subsonic_subway
Running analysis at 4715
Running analysis at 4746
Running analysis at 4818
Running analysis at 4848
Running analysis at 4920
Running analysis at 4951
Running analysis at 5022
Running analysis at 5053
Running analysis at 5124
Running analysis at 5155
Running analysis at 5226
Running analysis at 5257
Running analysis at 5328
Running analysis at 5359
Running analysis at 5430
Running analysis at 5501
Running analysis at 5532
Running analysis at 5603
Running analysis at 5634
Running analysis at 5705
Running analysis at 5736
Running analysis at 5807
Running analysis at 5838
Running analysis at 5909
Running analysis at 5940
Runn

KeyboardInterrupt: 